### Exercise 4 - Introduction to PyTorch (Solution)

In this exercise, we will introduce the fundamental concepts of PyTorch and show how this framework can be applied to build and train custom neural networks. 


**What is PyTorch?**\
PyTorch is a Python-based library designed for deep learning. It is distinguished by its dynamic computational graph, which enables researchers and developers to construct models with a high degree of flexibility. PyTorch has found extensive use in various scientific and engineering domains due to its ease of use and extensive research-friendly features.

This exercise is based on the [PyTorch 60-Minute Blitz](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html).

### 4.1 Tensors
Tensors are used to encode inputs and outputs of a model, as well as a model's parameters. They are comparable to NumPy's ndarrays, but they have the advantage that PyTorch tensors can run on GPUs and other accelerators. 

Extra ressources: [Intuition behind tensors](https://www.youtube.com/watch?v=f5liqUk0ZTw)

In [ ]:
import torch
import numpy as np

**4.1.1 Tensor Initialization**

Tensors can be initialized in multiple ways.

In [ ]:
# directly from data
data = [[1, 2], [3, 4]]
x_data = torch.tensor(data)
print(f"Tensor from data: \n {x_data} \n")

# from a NumPy array
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
print(f"Tensor from NumPy array: \n {x_np} \n")

# from another tensor
print("Shape given by another tensor:")
x_ones = torch.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")
x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

# with random or constant values
shape = (2, 3,)
print("Shape defined:", shape)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)
print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

**4.1.2 Tensor Attributes**

We can print information such as the tensor shape, the tensor datatype, and the device on which they are stored.

In [ ]:
tensor = torch.rand(3, 4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

**4.1.3 Tensor Operations**

For a full list of available tensor operations check out the corresponding [PyTorch documentation](https://pytorch.org/docs/stable/torch.html).

In [ ]:
# Moving tensor to the GPU if available
if torch.cuda.is_available():
    tensor = tensor.to('cuda')
    print(f"Device tensor is stored on: {tensor.device}")

In [ ]:
# standard numpy-like indexing and slicing
tensor = torch.ones(4, 4)
tensor[:,1] = 0
print(f"Second column replaced with zeros \n {tensor} \n")

---
<span style="color:orange">**EXERCISE**</span>: **Indexing and Slicing**

In [ ]:
tensor_exercise = torch.tensor([[1, 2, 3, 4],
                       [5, 6, 7, 8],
                       [9, 10, 11, 12],
                       [13, 14, 15, 16]])

# Exercise 1: Select the last 3 rows of the first 3 columns and assign them to 'submatrix'
# submatrix = ...
print("Extracted submatrix:\n", submatrix)

# Exercise 2: Replace the extracted submatrix with a tensor of zeros in the original tensor
# ...
print("Tensor after replacement:\n", tensor_exercise)

In [ ]:
# joining tensors (concatenation or stacking)
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(f"Concatenated tensor \n {t1} \n")

In [ ]:
# multiplying tensors
# element-wise product
print(f"tensor.mul(tensor) \n {tensor.mul(tensor)} \n")
# Alternative syntax:
print(f"tensor * tensor \n {tensor * tensor}")

# matrix multiplication
print(f"tensor.matmul(tensor.T) \n {tensor.matmul(tensor.T)} \n")
# Alternative syntax:
print(f"tensor @ tensor.T \n {tensor @ tensor.T}")

In [ ]:
# inplace operations have a trailing underscore
print(f"Tensor before inplace addition \n {tensor} \n")
tensor.add_(5)
print(f"Tensor after inplace addition \n {tensor} \n")

---
<span style="color:orange">**EXERCISE**</span>: **Element-wise Logarithm, multiplication and calculation of the mean**

In [ ]:
tensor = torch.tensor([1.0, 2.0, 3.0, 4.0])

# Exercise 1: Calculate the element-wise natural logarithm and multiply the result with the tensor itself
log_tensor = ...
print(log_tensor)

# Exercise 2: Calculate the mean of the resulting tensor
mean_log = ...
print(mean_log)

Note: In-place operations can be problematic when computing derivatives because of a loss of history. 

**4.1.4 Bridge with NumPy**

Tensors on the CPU and NumPy arrays can share their underlying memory locations, and a change to one will cause the other to change.

In [ ]:
t = torch.ones(5)
print(f"t: {t}")
# tensor to NumPy array
n = t.numpy()
print(f"n: {n}")

In [ ]:
t.add_(1)
# both arrays are modified
print(f"t: {t}")
print(f"n: {n}")

In [ ]:
n = np.ones(5)
# NumPy array to tensor
t = torch.from_numpy(n)

In [ ]:
np.add(n, 1, out=n)
# both arrays are modified
print(f"t: {t}")
print(f"n: {n}")

### 4.2 Introduction to torch.autograd

``torch.autograd`` is PyTorch's engine for automatic differentiation. It is essential for the training of neural networks.

**4.2.1 Differentiation in Autograd**

The argument ``required_grad=True`` signals to ``autograd`` that every operation on those tensors should be tracked. This allows ``autograd`` to collect gradients.

In [ ]:
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)
Q = 3*a**3 - b**2

If ``a`` and ``b`` are parameters of a neural networks the error function ``Q`` could looks like this:

\begin{align}Q = 3a^3 - b^2\end{align}

For the training of the neural network we need to calculate the gradients with respect to the parameters: 

\begin{align}\frac{\partial Q}{\partial a} = 9a^2\end{align}

\begin{align}\frac{\partial Q}{\partial b} = -2b\end{align}

With ``autograd`` you can calculate those gradients by calling ``.backward()`` on Q. The ``gradient`` argument is used here to specify how much the tensors ``a`` and ``b`` should influence the gradient calculation of ``Q``. By providing ``external_grad`` as the gradient argument, you ensure that both ``a`` and ``b`` are treated as if they contribute equally to the gradient of ``Q`` (both having a weight of 1.0).

In [ ]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

# check if collected gradients are correct
print(9*a**2 == a.grad)
print(-2*b == b.grad)

---
<span style="color:orange">**EXERCISE**</span>: **torch.autograd**

In [ ]:
x = torch.tensor([2.0, 3.0], requires_grad=True)
y = 2 * x[0] + 3 * x[1]

# Exercise 1: Calculate gradients
...

# Exercise 2: Print gradients of 'x'
print("Gradients of x:", ...)

**4.2.2 Computational Graph**

``autograd`` functions by maintaining a record of both data (tensors) and executed operations in a directed acyclic graph (DAG) composed of Function objects. Within this graph structure, the input tensors serve as the starting point (leaves), and the output tensors act as the endpoints (roots). By traversing this graph in a reverse manner, one can automatically compute gradients using the chain rule.

- **Forward pass**: ``autograd`` performs operations to compute the resulting tensor and maintains the operation's gradient function in the DAG.

- **Backward pass**: ``autograd`` (triggered by calling ``.backward()`` on the DAG root) computes the gradients from each ``.grad_fn``, accumulates them in the corresponding tensor's ``.grad`` attribute, and applies the chain rule to propagate gradients to the leaf tensors.

``autograd`` tracks operations for tensors with ``requires_grad=True``, while setting ``requires_grad=False`` excludes them; if any input tensor has ``requires_grad=True``, the output tensor will also require gradients.

Note: In PyTorch, DAGs (Directed Acyclic Graphs) are dynamic, and it's important to know that a new graph is built from scratch after each ``.backward()`` call. This flexibility enables the use of control flow statements and the ability to modify the model's shape, size, and operations in each iteration as required.


---
<span style="color:orange">**EXERCISE**</span>: **DAG**\
Think about the answers before executing the following code.


In [ ]:
x = torch.rand(5, 5)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad=True)

a = x + y
print(f"Does `a` require gradients?: {a.requires_grad}")
b = x + z
print(f"Does `b` require gradients?: {b.requires_grad}")

### 4.3 Neural Networks with PyTorch

The ``torch.nn`` package can be used to construct neural networks. ``nn.Module``s contain layers, and a method ``forward(input)`` that returns the ``output``.

Typical training procedure for a neural network:
1. Define neural network with some learnable parameters (weights)
2. Iterate over dataset of inputs
3. Process input through network
4. Compute the loss (how wrong is the output)
5. Backpropagate to calculate the gradient for each of the network's weights
6. Update the weights of the network

**4.3.1 Define the network**

Define the ``forward`` function. The ``backward`` function is is automatically defined. 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

# learnable parameters of the model
params = list(net.parameters())
print(f"Number of learnable parameters: {len(params)} \n")
print(params[0].size())  # conv1's .weight

In [ ]:
# random input
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

In [ ]:
# zero gradient buffers of all parameters and backprops with random gradients
net.zero_grad()
out.backward(torch.randn(1, 10))

**4.3.2 Loss Function**

The loss function computes a value that estimates how far away the output is from the target. For the full list of available loss functions check out the [PyTorch documentation](https://pytorch.org/docs/stable/nn.html#loss-functions).

Example: ``MSELoss`` (mean squared error (squared L2 norm))


In [ ]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

**4.3.3 Backprop**

To initiate error backpropagation, use ``loss.backward()``, but make sure to clear existing gradients; otherwise, the new gradients will accumulate onto the existing ones. This step is crucial for accurate gradient calculations.


In [ ]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

**4.3.4 Update the weights**

A simple update rule is the Stochastic Gradient Descent (SGD):

*weight = weight - learning_rate * gradient*

The ``torch.optim`` package implements different update rules such as SGB, Nesterov-SGD, Adam, RMSProp, etc.

In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

---
<span style="color:orange">**EXERCISE**</span>: **Neural Networks Recap**
1. What is the purpose of the forward method in a PyTorch neural network model?
2. How do you define a convolutional layer in PyTorch, and what does the nn.Conv2d module do?
3. In the provided code, how many learnable parameters does the neural network model Net have, and how can you access them?
4. Explain what happens when you call net.zero_grad() and why it is important.
5. What does the optimizer.step() method do, and when is it typically called in the training loop?

---
<span style="color:orange">**EXERCISE**</span>: **Custom Layer**\
Create  a custom neural network layer called CustomLayer that inherits from nn.Module. The layer should take an input tensor and compute the element-wise square of the input tensor.
1. Define the CustomLayer class with a forward method that performs the specified operation.
2. Instantiate the CustomLayer.
3. Generate a random input tensor with dimensions (1, 3, 4, 4).
4. Apply the CustomLayer to the input tensor to calculate the element-wise square.
5. Print the input tensor and the resulting output tensor.

In [ ]:
import torch
import torch.nn as nn

# 1. Define the CustomLayer class
# ...
# solution
class CustomLayer(nn.Module):
    def forward(self, x):
        return x**2

# 2. Instantiate the CustomLayer
...

# 3. Generate a random input tensor
...

# 4. Apply the CustomLayer to the input tensor
...

# 5. Print the input tensor and the resulting output tensor
print("Input Tensor:")
# print(input_tensor)
print("\nOutput Tensor (Element-wise Square):")
# print(output_tensor)


### 4.4 Training a Classifier (Optional)

When dealing with image, text, audio, or video data, you can use standard Python packages to load data into a NumPy array, which can then be converted into a ``torch.*Tensor``.

- For images: Packages like Pillow and OpenCV are useful.
- For audio: You can use packages such as scipy and librosa.
- For text: You have options like raw Python, Cython-based loading, NLTK, and SpaCy.

For image-related tasks, ``torchvision`` is a convenient package that provides data loaders for common datasets (e.g., ImageNet, CIFAR10, MNIST) and data transformers for images. This eliminates the need for writing repetitive code.

In this tutorial, we'll work with the CIFAR10 dataset, which includes classes like 'airplane,' 'automobile,' 'bird,' 'cat,' 'deer,' 'dog,' 'frog,' 'horse,' 'ship,' and 'truck.' The images in CIFAR-10 are 3-channel color images with dimensions 32x32 pixels.

Step:
1. Load and normalize CIFAR10 training and test datasets
2. Define Convolutional Neural Network (CNN)
3. Define loss function
4. Train network on training data
5. Test network on test data

**4.4.1 Load and normalize CIFAR10**

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
# transform PILImage images
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Note: If running on Windows and you get a BrokenPipeError, try setting the num_worker of torch.utils.data.DataLoader() to 0.

In [ ]:
# Show example images and labels
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

**4.4.2 Define Convolutional Neural Network**

Use neural network from section 4.3 and modify it to take 3-channel images (RGB instead of grayscale).

---
<span style="color:orange">**EXERCISE**</span>: **Defining a classifier**\
Fill in the gaps in the following code.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # define the first convolutional layer (conv1): 3 input channels, 6 output channels, kernel size: 5
        # self.conv1 = ...
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # apply the first convolutional layer, activation (relu), and pooling
        # x = ...
        x = self.pool(F.relu(self.conv1(x)))
        # apply the first convolutional layer, activation (relu), and pooling
        # x = ...
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

**4.4.3 Define loss function and optimizer**

---
<span style="color:orange">**EXERCISE**</span>: **Defining a loss function**\
Implement a cross entropy loss function

Hint: check out the [PyTorch documentation](https://pytorch.org/docs/stable/nn.html#loss-functions)

In [ ]:
import torch.optim as optim

# criterion = ...
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

**4.4.4 Train network**

---
<span style="color:orange">**EXERCISE**</span>: **Training a classifier**\
Fill in the gaps in the following code.

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        # ...

        # perform the forward pass
        # outputs = ...
        outputs = net(inputs)
        # calculate the loss ussing the criterion
        # loss = ...
        # perform the backward pass to compute gradients
        # ...
        # update the model's parameters using the optimizer
        # ...

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')


**4.4.5 Test network on test data**

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

This performance is significantly better than random chance, which would be a 10% accuracy when randomly selecting from 10 classes. It appears that the network has learned something meaningful.